In [39]:
import pandas as pd
import numpy as np

# 1. Portafoglio attuale del fondo 'CGE_FUND'
portfolio_data = {
    'ticker': ['AAPL', 'AAPL_BOND', 'VOW_DE', 'P911_DE', 'MSFT'],
    'market_value_eur': [1500000, 2100000, 9000000, 4500000, 2500000]
}
df_portfolio = pd.DataFrame(portfolio_data)

# 2. Anagrafica degli asset
assets_data = {
    'ticker': ['AAPL', 'AAPL_BOND', 'X_TICKER', 'VOW_DE', 'P911_DE', 'MSFT', 'GOOG'],
    'issuer_name': ['Apple Inc.', 'Apple Inc.', 'Azienda X', 'Volkswagen AG', 'Porsche AG', 'Microsoft Corp.', 'Alphabet Inc.'],
    'issuer_group': ['Apple Group', 'Apple Group', 'Gruppo X', 'Volkswagen Group', 'Volkswagen Group', 'Microsoft Group', 'Alphabet Group'],
    'is_ucits_eligible': [True, True, True, True, True, True, True]
}
df_assets_master = pd.DataFrame(assets_data)

# 3. Dati del fondo
fund_data = {
    'fund_id': ['CGE_FUND'],
    'nav_eur': [100000000] 
}
df_fund = pd.DataFrame(fund_data)

In [120]:

def check_ucits_compliance(ticker_to_add: str, amount_to_add_eur: float, df_portfolio : pd.DataFrame = df_portfolio, df_assets_master: pd.DataFrame = df_assets_master, df_fund: pd.DataFrame = df_fund):
    if not ticker_to_add in df_assets_master['ticker'].values:
        raise ValueError(f"Ticker not found")
    if not df_assets_master[df_assets_master['ticker'] == ticker_to_add]['is_ucits_eligible'].iloc[0]:
        return "Not Eligible Asset"
       
    current_nav = df_fund.loc[0, 'nav_eur']
    new_nav = current_nav + amount_to_add_eur # let's suppose the liabilities remain the same
    df_new_portfolio = df_portfolio.copy()
    # add ticker and value if not already in the portfolio, otherwise just add the amount
    if ticker_to_add in df_new_portfolio['ticker'].values:
        index = df_new_portfolio.loc[df_new_portfolio['ticker'] == ticker_to_add].index[0]
        df_new_portfolio.loc[index, 'market_value_eur'] += amount_to_add_eur
    else:
        df_new_portfolio.loc[len(df_new_portfolio)] = [ticker_to_add, amount_to_add_eur]
    warning_list=[] #collector of all warnings
    df_portfolio_info = pd.merge(df_new_portfolio, df_assets_master, how='left', on='ticker').drop('is_ucits_eligible', axis=1)
    #get issuer and group
    issuer = df_assets_master[df_assets_master['ticker'] == ticker_to_add]['issuer_name'].iloc[0]
    group = df_assets_master[df_assets_master['ticker'] == ticker_to_add]['issuer_group'].iloc[0]
    #get issuer value and group value
    market_value_total_issuer = df_portfolio_info[df_portfolio_info['issuer_name'] == issuer]['market_value_eur'].sum()
    market_value_total_group  = df_portfolio_info[df_portfolio_info['issuer_group'] == group]['market_value_eur'].sum()
    #10% rule
    if market_value_total_issuer/new_nav > 0.1:
        warning_list.append(f"10% limit breached for the issuer: {issuer}. The value is now: {market_value_total_issuer/new_nav}")
    # 20% rule
    if market_value_total_group/new_nav > 0.2:
        warning_list.append(f"20% limit breached for the group: {group}. The value is now: {market_value_total_group/new_nav}")

    
    df_issuers = df_portfolio_info.groupby('issuer_name')['market_value_eur'].sum()
    df_issuers = df_issuers.divide(new_nav / 100).reset_index(name='percentage')
    forty_rule = df_issuers[df_issuers["percentage"] >= 5]["percentage"].sum()
    #40% rule
    if forty_rule > 40:
        warning_list.append(f"40% limit breached for the considered fund. The value is now: {forty_rule}")


    if not warning_list:
        return (True, ["Transaction Approved"])
    else:
        return (False, warning_list)
    #current_mar_value =  df_assets_master[df_assets_master['ticker'] == ticker_to_add].loc[0, 'is_ucits_eligible']
    
   
    
    
    # return(df_portfolio['market_value_eur'].sum())

    
    
    

In [121]:
print(check_ucits_compliance(ticker_to_add='P911_DE', amount_to_add_eur = 15000000000, df_portfolio= df_portfolio, df_assets_master = df_assets_master, df_fund= df_fund))

99.36754966887418
       issuer_name  percentage
0       Apple Inc.    0.023841
1  Microsoft Corp.    0.016556
2       Porsche AG   99.367550
3    Volkswagen AG    0.059603
['10% limit breached for the issuer: Porsche AG. The value is now: 0.9936754966887418', '20% limit breached for the group: Volkswagen Group. The value is now: 0.994271523178808', '40% limit breached for the considered fund. The value is now: 99.36754966887418']
